# Chapter8 텍스트 분석 ~04 - 실리콘밸리 5차 정기세션 과제

## 스팸 텍스트 메시지 데이터 활용 링크: https://www.kaggle.com/team-ai/spam-text-message-classification/download

### 1. 텍스트 분석 수행 프로세스 및 텍스트 정규화 작업을 정리하세요

<텍스트 분석 수행 프로세스>
1.텍스트 사전 준비작업 : 텍스트를 피처로 만들기 전에 사전에 클렌징, 대/소문자 변경, 특수문자 삭제 등의 클렌징 작업, 단어등의 토근화 작업, 의미없는 단어 제거 작업, 어근 추출등의 텍스트 정규화 작업을 수행
2.피처 벡터화/추출: 사전 준비 작업으로 가공된 텍스트에서 피처를 추출하고 여기에 벡터 값을 할당합니다. 
3.피처 벡터화된 데이터 세트에 ML 모델을 적용해 학습/예측 및 평가를 수행

<텍스트 정규화 작업>
* 클렌징
* 토큰화
* 필터링/스톱 워드 제거/철자 수정
* Stemming
* Lemmatization

In [2]:
from nltk import sent_tokenize
from nltk import word_tokenize
import pandas as pd
import numpy as np

### 2. 해당 데이터의 index가 901인 문장에 대해 문장별 단어 토큰화(교재 함수 사용)를 수행하고 스톱 워드를 제거하세요

In [22]:
text_spam=pd.read_csv("SPAM text message 20170820 - Data.csv")
text_spam.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [23]:
text_sample= text_spam.iloc[901,1]
text_sample = str(text_sample)

In [10]:
def tokenize_text(text):
     
    sentences = sent_tokenize(text)
    
    word_tokens = [word_tokenize(sentence) for sentence in sentences]
    return word_tokens

In [12]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')
word_tokens = tokenize_text(text_sample)
all_tokens = []
for sentence in word_tokens:
    filtered_words=[]
    for word in sentence:
        word = word.lower()
        if word not in stopwords:
            filtered_words.append(word)
    all_tokens.append(filtered_words)
    
print(all_tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jaeyong\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jaeyong\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


[['probably', 'money', 'worries', '.'], ['things', 'coming', 'due', 'several', 'outstanding', 'invoices', 'work', 'two', 'three', 'months', 'ago', '.']]


### 3.  해당 문장의 결과 토큰화 값들 중 진행형에 대해 Stemmimg과 Lemmatization을 수행 후 결과를 비교하세요

In [13]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()
print(stemmer.stem('outstanding'))

outstand


In [16]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemma = WordNetLemmatizer()
print(lemma.lemmatize('outstanding','v'))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jaeyong\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


outstanding


### 4. 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가
### 4-1. 위의 스팸 데이터로 train, test set을 나누세요 (힌트: train_test_split)

In [39]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(text_spam['Message'], text_spam['Category'], test_size=0.3, random_state=0)

### 4-2. 두 개의 피처 벡터화를 수행 후 각각의 벡터화 shape을 구하세요

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer()
cnt_vect.fit(x_train)
X_train_cnt_vect = cnt_vect.transform(x_train)

X_test_cnt_vect = cnt_vect.transform(x_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (3900, 7309)


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(x_train)
X_train_tfidf_vect = tfidf_vect.transform(x_train)
X_test_tfidf_vect = tfidf_vect.transform(x_test)
print('학습 데이터 Text의 TF-IDF Shape:',X_train_tfidf_vect.shape)

학습 데이터 Text의 TF-IDF Shape: (3900, 7309)


### 4-3. 각각에 로지스틱 회귀 모형을 적용해 예측해 보세요

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는 0.977


C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [37]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.970


## Women's E-Commerce Clothing Reviews 데이터 링크: https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

### 5. 파이프라인 사용 
### 5-1. 새로운 데이터 셋을 다운로드 후 'Review text'로 'Rating'을 예측하도록 적절한 전처리를 수행 후 test, train set으로 분할하세요(힌트: 결측치 처리-어떻게?)

In [65]:
e_reivew=pd.read_csv("Womens Clothing E-Commerce Reviews.csv")
e_reivew.head()
e_review=e_reivew[['Review Text', 'Rating']]
e_review.head()
e_review.isna().sum()
na_index = e_review[e_review["Review Text"].isna()].index
e_review = e_review.drop(na_index)
e_review.isna().sum()

Review Text    0
Rating         0
dtype: int64

In [66]:
X_train, X_test, y_train, y_test = train_test_split(e_review['Review Text'],e_review['Rating'], test_size=0.3, random_state=42)
print(X_train)

22502    I got the cream color and absolutely love the ...
392      If your hips are bigger than a size 6 us you c...
372      This has great drape, length, the pattern is s...
16747    When i read all these positive reviews i did n...
16697    I love this skirt. the material is soft and be...
                               ...                        
12408    I bought the grey, which is a gorgeous color. ...
22387    Love love this dress! i got the regular size, ...
5590     I bought one of these tees and i washed it a f...
901      I was excited to see these jeans since they ca...
16401    I bought this sweater and love it! one of my f...
Name: Review Text, Length: 15848, dtype: object


### 5-2. 사이킷런의 Pipeline방식을 적용해 TF-IDF기반으로 벡터화하고 로지스틱 회귀 모형으로 예측해보세요(파라미터 튜닝은 디폴트로 해도 되고 맘대로 해보세요)

In [67]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])
 
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\jaeyong\Anaconda3\envs\cuda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline을 통한 Logistic Regression 의 예측 정확도는 0.619
